In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load the revised dataset
df = pd.read_csv("CO2_emmisions_rev.csv")

# Identify the year columns (e.g., 1960–2023)
year_cols = sorted([c for c in df.columns if c.isdigit()], key=int)
years = np.array(list(map(int, year_cols)))

# Use only real countries
countries = df[df["Is_Country"] == "YES"]

# Use the existing "World" row if present, otherwise sum all countries
world_row = df.loc[df["Country Name"] == "World", year_cols]
if not world_row.empty and world_row.iloc[0].notna().any():
    world_vals = pd.to_numeric(world_row.iloc[0], errors="coerce").to_numpy(float)
else:
    world_vals = pd.to_numeric(countries[year_cols].T.sum(), errors="coerce").to_numpy(float)

# Build DataFrame and determine color for each year
g = pd.DataFrame({"Year": years, "CO2_kt": world_vals}).dropna(subset=["CO2_kt"])
g["Color"] = np.where(g["CO2_kt"].diff() > 0, "red", "green")
g.loc[g.index[0], "Color"] = "red"

# Plot
fig = go.Figure([
    go.Bar(
        x=g["Year"],
        y=g["CO2_kt"],
        marker_color=g["Color"],
        hovertemplate="Year=%{x}<br>CO₂ (kt)=%{y:,.0f}<extra></extra>"
    )
])
fig.update_layout(
    title="Annual Global CO₂ Emissions (1960–2023)",
    xaxis_title="Year",
    yaxis_title="Emissions (kt)",
    bargap=0
)

fig.show()



In [ ]:
# 5-year moving average
# Reuse the global annual series from the first chart
global_df = g[["Year", "CO2_kt"]].copy()

global_df["MA5"] = global_df["CO2_kt"].rolling(window=5, center=True, min_periods=3).mean()

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=global_df["Year"], y=global_df["CO2_kt"], marker_color="lightgray", opacity=0.5))
fig2.add_trace(go.Scatter(x=global_df["Year"], y=global_df["MA5"], mode="lines", line=dict(width=3), name="5-year moving average"))
fig2.update_layout(title="Global CO₂: Raw Annual vs 5-year Moving Average", xaxis_title="Year", yaxis_title="Emissions (kt)")
fig2.show()

# Decadal growth computation
def decade_blocks(start, end):
    return [(y, min(y + 9, end)) for y in range(start, end + 1, 10)]

blocks = decade_blocks(int(global_df["Year"].min()), int(global_df["Year"].max()))
dec_labels, dec_changes = [], []
vals = dict(zip(global_df["Year"], global_df["CO2_kt"]))
for a, b in blocks:
    if a in vals and b in vals:
        dec_labels.append(f"{a}s")
        dec_changes.append(vals[b] - vals[a])

fig3 = go.Figure()
fig3.add_trace(go.Bar(x=dec_labels, y=dec_changes, marker_color=np.where(np.array(dec_changes)>=0, "red", "green")))
fig3.update_layout(title="Decadal Change in Global CO₂ (end minus start)", xaxis_title="Decade", yaxis_title="Change (kt)")
fig3.show()


In [ ]:

countries = ["China", "United States", "India", "Russian Federation", "European Union", "World"]

# Helper to return (years, values) for a given country name
def series_for(country_name):
    # df and year_cols are already defined earlier in the notebook
    row = df.loc[df["Country Name"] == country_name, year_cols]
    if row.empty:
        return None, None

    ys = np.array(list(map(int, year_cols)))
    vals = pd.to_numeric(row.iloc[0], errors="coerce").to_numpy(float)
    return ys, vals
fig4 = go.Figure()
for c in countries:
    ys, vals = series_for(c)
    if vals is not None and not np.all(np.isnan(vals)):
        data = pd.DataFrame({"Year": ys, "CO2_kt": vals}).dropna()
        data = data[data["Year"] >= 1990]  # filter from 1990 onward
        fig4.add_trace(go.Scatter(x=data["Year"], y=data["CO2_kt"], mode="lines", name=c))

fig4.update_layout(
    title="CO₂ Emissions: Top-5 Countries vs World (1990–Present)",
    xaxis_title="Year",
    yaxis_title="Emissions (kt)"
)
fig4.show()


In [ ]:
last10 = global_df.tail(10).dropna(subset=["CO2_kt"])
x = np.arange(len(last10))
m, b = np.polyfit(x, last10["CO2_kt"], 1)
trend = m * x + b

fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=last10["Year"], y=last10["CO2_kt"], mode="markers+lines", name="Global CO₂"))
fig5.add_trace(go.Scatter(x=last10["Year"], y=trend, mode="lines", name=f"Linear Fit (slope={m:,.0f} kt/yr)", line=dict(dash="dash")))
fig5.update_layout(title="Latest 10-Year Global CO₂ Trend", xaxis_title="Year", yaxis_title="Emissions (kt)")
fig5.show()
